# Auto-STED for imaging of FISH spot pairs

## How-To run:

1) Run the cell bellow ONCE if you (re)started the notebook

(to run a cell, click in it and press Ctrl-Enter)

In [3]:
from specpy import *
from Util.coordinates import *
from Util.datastructures import *
from Util.tile_util import *
from Util.imspector_util import *
from Util.dot_detection.Fiji_coordinate_detection import *
from Util.coordinate_util import *
from Util.dot_detection.Fiji_dummy import *
from Util.dot_detection.Fiji_coordinate_detection import read_coords
import pprint
from Util import datastructures, imspector_util
import time
from matplotlib import pyplot as plt
from skimage.feature import blob_dog, blob_log, blob_doh
from scipy import ndimage, spatial, stats
from spot_util import *
from display_util import *
%matplotlib inline


# Actual measurement
2) insert the name of the results folder in the cell below (line 2)
3) run the cell (Imspector has to be running and Tools->Run server has to be ckecked in Imspector)

# Stopping the run
4) click the "STOP"-button at the top of this window
5) the current measurement in Imspector will finish, but won't be saved

In [30]:
# make results dir
f = 'C:/Users//RESOLFT/Desktop/AUTOMATION/GM_HS2HS3_4min_prep(16-08-23)/'
d = os.path.dirname(f)
if not os.path.exists(d):
    os.makedirs(d)

im = Imspector()

see = Settings()
see.load_from_file('C:/Users//RESOLFT/Desktop/ov.json')
coords = generate_grid_oop([0,0],[2e-3, 2e-3], [5e-5, 5e-5])
for c in coords:
    c.set_fov_len([5e-05, 5e-05, 1e-5])
   

#keep track of maximum z -> use that as next midplane
zpos = 0.0;
    
nm = datastructures.NameManagement(d)
nm.add_counter('field')
nm.add_counter('sted')

# measurement will be created on first run
ms = None

for c in coords:
    
    offset_old = c.get_scan_offset()
    offset_old[2] = zpos
    c.set_offset_coords(offset_old)
    
    see.set_to_coordinates(c)
    
    
    name = nm.get_next_image_name('field')
    acquire_measurement(im, see, ms)
    ms = im.active_measurement()
    
    # JUST DO THE MOVES
    #im.close(ms)
    #continue
    
    sites = pair_finder(ms, threshold = 0.8)
    
    # get images in both channels
    # and plot the detections in max-projection
    stack1 = ms.stack(0).data()[0,:,:,:]
    stack2 = ms.stack(1).data()[0,:,:,:]
    draw_detections_2c(stack1, stack2, sites, [1, 10], 0, 3)
    plt.show()
    
    co = c.copy()
    
    corner = middle2corner(c.get_scan_offset(), c.get_fov_len())
    pixelsd = get_pixel_size(ms)
    actual_coords = corner2spot(corner, sites, pixelsd)
    
    # update focus
    zpos += focus_in_stack(ms.stack(0).data(), pixelsd[2])    
    
    ms.save_as(name)
    
    #im.close(ms)
    
    print('new z:' + str(zpos))
    print(sites)
    
    see2 = Settings()
    see2.load_from_file('C:/Users//RESOLFT/Desktop/det.json')
    
    nm.reset_counter('sted')
    
    for i in actual_coords:
        co.set_fov_len([3e-06, 3e-06, 1e-06])
        co.set_offset_coords(i)
        see2.set_to_coordinates(co)
        
        name = nm.get_next_image_name('sted')
        acquire_measurement_dummy(im, see2, ms)
        
        ms = im.active_measurement()
        ms.save_as(name)
        #im.close(ms)

new z:-3e-06
[[230, 71, 5], [135, 149, 8]]
new z:-3.5e-06
[[289, 254, 24], [77, 9, 29]]
new z:-2.75e-06
[[72, 64, 17], [194, 87, 18], [220, 264, 18], [67, 60, 21], [228, 71, 27], [68, 291, 32]]
new z:-1.75e-06
[[109, 288, 16], [224, 205, 25]]
new z:-1.5e-06
[[119, 27, 23], [209, 260, 27]]
new z:-1.5e-06
[[103, 249, 12], [215, 77, 24], [198, 234, 29]]
new z:-1e-06
[[127, 74, 11], [295, 168, 27]]
new z:-5e-07
[[128, 84, 10], [221, 280, 19], [219, 284, 19], [82, 112, 23], [295, 283, 25], [278, 309, 26]]
new z:-2.5e-07
[[179, 300, 21], [119, 214, 23], [284, 203, 25], [82, 35, 31], [280, 218, 31], [246, 134, 33]]
new z:1e-06
[[56, 98, 13], [135, 72, 16], [299, 124, 26], [154, 48, 34]]
new z:1.5e-06
[[312, 32, 14], [157, 142, 17], [248, 125, 25], [183, 90, 29], [144, 284, 30], [118, 52, 31]]
new z:2e-06
[[78, 107, 16], [76, 115, 16], [276, 28, 19], [255, 314, 25], [168, 290, 32]]
new z:2.75e-06
[]
new z:2.75e-06
[[118, 80, 17], [123, 205, 22]]
new z:2.75e-06
[[77, 320, 20], [86, 282, 24], [2

KeyboardInterrupt: 

# code below here is only for testing purposes

In [20]:
stack1 = ms.stack(0).data()[0,:,:,:]
stack2 = ms.stack(1).data()[0,:,:,:]

sig = 3 / np.sqrt(2)
dets1 = detect_blobs(stack1, [sig, sig, sig], 0.002)    
dets1=list(dets1)
dets2 = detect_blobs(stack2, [sig, sig, sig], 0.002)    
dets2=list(dets2)

In [17]:
actual_coords

[[-1.54e-05, -1.615e-05, 1.9999999999999995e-06],
 [1.1449999999999997e-05, 7.549999999999997e-06, 4.8e-06]]

In [20]:
co

In [2]:
pair_finder(Imspector().active_measurement())

[[23, 313, 2], [321, 22, 5], [323, 30, 5], [95, 180, 9]]